### Importar librerías

In [1]:
# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

from src.soporte import get_locations

---

### Tabla `municipios`
* Trasponemos
* Le ponemos el id (pk)

### Tabla `tiempo` 
* Localización solo id (fk)
* Nueva tabla `cielo`con cada tipo de cielo y reemplazamos `tiempo`
* Reset index para (pk) y fecha en columna
* pd.to_datetime a la fecha

### Tabla `ubicaciones`
* Eliminamos el link
* Reemplazamos el municipio por el id
* Nueva tabla `category`
* Nueva table `closed bucket`
* fsq_id (pk)
* reemplazar municipio (fk)

---

`Tiempo`

In [2]:
df_tiempo = pd.read_csv('data/df_aemet_final.csv', index_col=0)

In [3]:
df_tiempo.head(1)

,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
fecha,,,,,,,,,,,,,,
2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


In [4]:
# Comprobamos que no hay duplicados
df_tiempo.duplicated().value_counts()

False    7884
True        1
Name: count, dtype: int64

In [5]:
# Eliminamos duplicados
df_tiempo.drop_duplicates(inplace=True)

In [6]:
df_tiempo.reset_index(inplace=True)
# Habrá que hacer un pd.to_datetime

In [7]:
# Construimos un df auxiliar para los id de los municipios
df_aux = df_tiempo['localizacion_id'].str.split('-id', expand=True)
df_aux = df_aux.drop_duplicates().set_index(0)
# Creamos un diccionario para convertir
dc_municipios = df_aux.to_dict()[1]
df_aux.head(1)

,1
0,
acebeda-la,28001


In [8]:
# Reemplazamos los id
df_tiempo['municipio_id'] = df_tiempo['localizacion_id'].str.split('-id', expand=True)[1].astype(int)

In [9]:
# Ya no necesitamos la columna de localización
df_tiempo.drop(columns='localizacion_id', inplace=True)

Tabla cielo

In [10]:
# Dataframe (series) del cielo, eliminando duplicados y dejando los índices adecuados
df_cielo = df_tiempo['cielo'].drop_duplicates().reset_index(drop = True)

# Creamos un diccionario e invertimos las claves y valores
dc_cielo = {v: k for k, v in df_cielo.to_dict().items()}

# Creamos lo que será nuestra tabla de cielo
df_cielo = df_cielo.reset_index().rename(columns={'index': 'cielo_id', 'cielo': 'estado'})

In [11]:
# Cambiamos los valores en el df de tiempo
df_tiempo['cielo'] = df_tiempo['cielo'].apply(lambda x: dc_cielo[x])
# Renombramos la columna
df_tiempo.rename(columns={'cielo': 'cielo_id'}, inplace = True)

---

`Municipios`

In [12]:
df_municipios = pd.read_csv('data/municipios.csv', index_col=0)

In [13]:
# Tenemos que trasponer el DataFrame para que sea más cómodo
df_municipios = df_municipios.T
df_municipios.head(1)

,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda


In [14]:
# Comprobamos que no hay duplicados
df_municipios.duplicated().value_counts()

False    176
Name: count, dtype: int64

Tenemos un problema, a la lista de municipios de este dataframe le faltan algunos municipios del dataframe tiempo

In [15]:
# Obtenemos la lista de municipios en df_municipios
lista_municipios = list(df_municipios.index)
# Obtenemos la lista de municipios en df_tiempo, que previamente habíamos formateado en df_aux
lista_municipios_tiempo = list(df_aux.index)

In [16]:
# Vemos que la lista de municipios está contenida en la otra
set(lista_municipios).issubset(set(lista_municipios_tiempo))

True

In [17]:
# Comprobamos cuáles son los que faltan
lista_municipios_faltantes = list(set(lista_municipios_tiempo).difference(set(lista_municipios)))
lista_municipios_faltantes

['cabrera-la', 'leganes', 'molinos-los']

In [18]:
# Utilizamos geopy para recuperar las coordenadas de los municipios faltantes
df_municipios_faltantes = pd.DataFrame(get_locations(lista_municipios_faltantes))

100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


In [19]:
df_municipios.sample()

,0,1,2
san-sebastian-de-los-reyes,40.5473698,-3.6260586,San Sebastián de los Reyes


In [20]:
# Ahora tenemos que dejar el dataframe con la misma estructura que el otro para poder concatenarlos
df_municipios_faltantes.set_index(['Nombre'], inplace=True)
# Adicionalmente extraemos el nombre de los municipios a partir de la dirección
df_municipios_faltantes[2] = df_municipios_faltantes[2].str.extract(r'(^[^,]+),')

In [21]:
# Añadimos estos municipios faltantes al dataframe original
df_municipios = pd.concat([df_municipios, df_municipios_faltantes]).sort_index()

In [22]:
# Renombramos para que tenga más sentido
df_municipios = df_municipios.rename(columns={0: 'latitud', 1: 'longitud', 2: 'nombre'})
df_municipios.head(1)

,latitud,longitud,nombre
acebeda-la,41.0869583,-3.624399,La Acebeda


In [23]:
df_municipios = df_municipios.reset_index(names='municipio_id')
df_municipios.head(1)

,municipio_id,latitud,longitud,nombre
0,acebeda-la,41.0869583,-3.624399,La Acebeda


Ahora en `df_municipios` cambiamos los `municipio_id` por sus valores

In [24]:
df_municipios['municipio_id'] = df_municipios['municipio_id'].apply(lambda x: dc_municipios[x])

---

`Ubicaciones`

In [25]:
df_ubicaciones = pd.read_csv('data/api_foursquare.csv', index_col=0)
df_ubicaciones.head(1)

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


In [26]:
# Comprobamos si hay duplicados en el fsq_id (pueden no serlo por municipio)
df_ubicaciones['fsq_id'].duplicated().value_counts()

fsq_id
False    822
True      85
Name: count, dtype: int64

In [27]:
df_ubicaciones.drop_duplicates(subset=['fsq_id'], keep='first', inplace=True)

Nos interesan todas las columnas salvo el link, que contiene información redundante con fsq_id

In [28]:
# Eliminamos la columna que nos sobra
df_ubicaciones = df_ubicaciones.drop(columns='link')

In [29]:
df_ubicaciones.head(1)

,municipio,category,fsq_id,closed_bucket,distance,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


Creamos la tabla de categorías

In [30]:
# Dataframe (series) de las categorías, eliminando duplicados y dejando los índices adecuados
df_categorias = df_ubicaciones['category'].drop_duplicates().reset_index(drop = True)

# Creamos un diccionario e invertimos las claves y valores
dc_categorias = {v: k for k, v in df_categorias.to_dict().items()}

# Creamos lo que será nuestra tabla de categorías
df_categorias = df_categorias.reset_index().rename(columns={'index': 'category_id'})

In [31]:
# Cambiamos los valores en el df de ubicaciones
df_ubicaciones['category'] = df_ubicaciones['category'].apply(lambda x: dc_categorias[x])
# Renombramos la columna
df_ubicaciones.rename(columns={'category': 'category_id'}, inplace = True)

Creamos la tabla closed_bucket

In [32]:
# Dataframe (series) de las categorías, eliminando duplicados y dejando los índices adecuados
df_closed_bucket = df_ubicaciones['closed_bucket'].drop_duplicates().reset_index(drop = True)

# Creamos un diccionario e invertimos las claves y valores
dc_closed_bucket = {v: k for k, v in df_closed_bucket.to_dict().items()}

# Creamos lo que será nuestra tabla de closed bucket
df_closed_bucket = df_closed_bucket.reset_index().rename(columns={'index': 'closed_bucket_id'})

In [33]:
# Cambiamos los valores en el df de ubicaciones
df_ubicaciones['closed_bucket'] = df_ubicaciones['closed_bucket'].apply(lambda x: dc_closed_bucket[x])
# Renombramos la columna
df_ubicaciones.rename(columns={'closed_bucket': 'closed_bucket_id'}, inplace = True)

Nos falta cambiar el municipio por el id correspondiente

In [34]:
df_ubicaciones['municipio'] = df_ubicaciones['municipio'].apply(lambda x: dc_municipios[x])
# Renombramos la columna
df_ubicaciones.rename(columns={'municipio': 'municipio_id'}, inplace = True)

---

### Creación de queries para crear las tablas

Queries de creación

In [35]:
# Crear la tabla Municipios
query_creacion_municipios = """
create table if not exists municipios (
    municipio_id INT primary key,
    latitud VARCHAR(20) not null,
    longitud VARCHAR(20) not null,
    nombre VARCHAR(50) not null unique
);
"""

# Crear la tabla Tiempo
query_creacion_tiempo = """
create table if not exists tiempo (
    tiempo_id SERIAL primary key,
    fecha DATE not null,
    cielo_id INT not null,
    temperatura_C INT not null,
    sensacion_termica_C INT not null,
    racha_maxima_kmh DECIMAL(10, 4) not null,
    precipitacion_mm DECIMAL(10, 4) not null,
    nieve_mm DECIMAL(10, 4) not null,
    humedad_relativa INT not null,
    prob_precip VARCHAR(10) not null,
    prob_nieve INT not null,
    prob_tormenta INT not null,
    avisos VARCHAR(20) not null,
    dir_viento VARCHAR(2) not null,
    vel_viento INT not null,
    municipio_id INT not null,
    foreign key (municipio_id) references municipios(municipio_id),
    foreign key (cielo_id) references cielo(cielo_id)
);
"""

# Crear la tabla Ubicaciones
query_creacion_ubicaciones = """
create table if not exists ubicaciones (
    municipio_id INT not null,
    category_id INT not null,
    fsq_id VARCHAR(50) primary key,
    closed_bucket_id INT not null,
    distance DECIMAL(10, 2) not null,
    name VARCHAR(100) not null,
    address VARCHAR(300) not null,
    latitude DECIMAL(10, 5) not null,
    longitude DECIMAL(10, 5) not null,
    foreign key (municipio_id) references municipios(municipio_id),
    foreign key (category_id) references categorias(category_id),
    foreign key (closed_bucket_id) references closed_bucket(closed_bucket_id)
);
"""

# Crear la tabla Cielo
query_creacion_cielo = """
create table if not exists cielo (
    cielo_id INT unique not null,
    estado VARCHAR(100) unique not null
);
"""

# Crear la tabla Categorías
query_creacion_categorias = """
create table if not exists categorias (
    category_id INT unique not null primary key,
    category VARCHAR(100) unique not null
);
"""

# Crear la tabla Closed bucket
query_creacion_closed_bucket = """
create table if not exists closed_bucket (
    closed_bucket_id INT unique not null primary key,
    closed_bucket VARCHAR(100) unique not null
);
"""

# En orden para crear las que tienen foreign key al final
lista_queries = [query_creacion_municipios, 
                 query_creacion_categorias, 
                 query_creacion_closed_bucket, 
                 query_creacion_cielo, 
                 query_creacion_tiempo,
                 query_creacion_ubicaciones]

Ejecución de queries de creación

In [36]:
# Conexión a la base de datos

try:
    conexion = psycopg2.connect(
        database = 'atrezzo',
        user = 'my_user',
        password = 'admin',
        host = 'localhost',
        port = '5432'
    )

except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('La contraseña es incorrecta')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('Error de conexión')
    else:
        print(f'Hay un error: {e}')

In [37]:
# Creamos un cursor
cursor = conexion.cursor()
# Ejecutamos la query de creación de tabla

for query in lista_queries:
    cursor.execute(query)

conexion.commit()
# Cerrar la conexión
conexion.close()

Ejecución de la inserción

In [38]:
query_insercion_municipios = """
INSERT INTO municipios (municipio_id, latitud, longitud, nombre)
VALUES
(%s, %s, %s, %s)
"""

query_insercion_tiempo = """
INSERT INTO tiempo (fecha, 
                    cielo_id, 
                    temperatura_C, 
                    sensacion_termica_C, 
                    racha_maxima_kmh, 
                    precipitacion_mm, 
                    nieve_mm, 
                    humedad_relativa,
                    prob_precip,
                    prob_nieve,
                    prob_tormenta,
                    avisos,
                    dir_viento,
                    vel_viento,
                    municipio_id)
VALUES
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

query_insercion_ubicaciones = """
INSERT INTO ubicaciones (municipio_id,
                    category_id,
                    fsq_id,
                    closed_bucket_id,
                    distance,
                    name,
                    address,
                    latitude,
                    longitude)
VALUES
(%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

query_insercion_cielo = """
INSERT INTO cielo (cielo_id, estado)
VALUES
(%s, %s)
"""

query_insercion_categorias = """
INSERT INTO categorias (category_id, category)
VALUES
(%s, %s)
"""

query_insercion_closed_bucket = """
INSERT INTO closed_bucket (closed_bucket_id, closed_bucket)
VALUES
(%s, %s)
"""

In [39]:
valores_municipios = [tuple(fila) for fila in df_municipios.values]

valores_categorias = [tuple(fila) for fila in df_categorias.values]

valores_cielo = [tuple(fila) for fila in df_cielo.values]

valores_closed_bucket = [tuple(fila) for fila in df_closed_bucket.values]

valores_tiempo = [tuple(fila) for fila in df_tiempo.values]

valores_ubicaciones = [tuple(fila) for fila in df_ubicaciones.values]

In [40]:
# Conexión a la base de datos

try:
    conexion = psycopg2.connect(
        database = 'atrezzo',
        user = 'my_user',
        password = 'admin',
        host = 'localhost',
        port = '5432'
    )

except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('La contraseña es incorrecta')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('Error de conexión')
    else:
        print(f'Hay un error: {e}')

In [41]:
# Creamos un cursor
cursor = conexion.cursor()
cursor.executemany(query_insercion_municipios, valores_municipios)
cursor.executemany(query_insercion_categorias, valores_categorias)
cursor.executemany(query_insercion_cielo, valores_cielo)
cursor.executemany(query_insercion_closed_bucket, valores_closed_bucket)
cursor.executemany(query_insercion_tiempo, valores_tiempo)
cursor.executemany(query_insercion_ubicaciones, valores_ubicaciones)
conexion.commit()
# Cerrar la conexión
conexion.close()